In [1]:
import numpy as np
import fasttext
import pandas as pd
import random
import itertools
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from resources.fasttext_transformer import classify
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
import copy
import scipy
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve
import gc
from sklearn.metrics import DetCurveDisplay, RocCurveDisplay
from resources.expansion import Expand_U
from resources.expansion import Expand_UW
import math

[nltk_data] Downloading package wordnet to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
model_2 = fasttext.load_model("models/ft_unsupervised_N_2.bin")
model_3 = fasttext.load_model("models/ft_unsupervised_N_3.bin")

In [3]:
df_sample = pd.read_csv('datasets/random_sample_data.csv')

In [4]:
# remove unnecessary comments
df_sample = df_sample.drop([x for x in range(len(df_sample)) if int(
    df_sample['label'][x]) != 0 and int(df_sample['label'][x]) != 1])

df_sample.tail()

,raw_comment,tokenized_comment,label
4477,all the flags are sikh religious flags. they d...,all the flag be sikh religious flag they do no...,1
4478,Salute the farmers,salute the farmer,0
4479,Let's also remember how many farmers lost thei...,let be also remember how many farmer lose thei...,0
4480,ABP news you guys dont have heart 8 farmers go...,abp news you guy dont have heart 8 farmer get ...,0
4481,These new laws actually give the power to the ...,these new law actually give the power to the f...,1


In [5]:
comments = df_sample['tokenized_comment']
labels = df_sample['label']

### Weak Supervision

In [6]:
def n_gram_count(sentence: str, n_gram_phrase: str):
    return sentence.count(n_gram_phrase)


def IDF(corpus, unique_words):
   idf_dict = {}
   N = len(corpus)
   for i in unique_words:
     count = 0
     for sen in corpus:
       if n_gram_count(sen,i) != 0:
         count = count+1
       idf_dict[i] = (math.log((1+N)/(count+1)))+1
   return idf_dict


def TF(sentence, n_gram_phrase, unique_words):
    """count of t in d / number of words in d

       each phrase in unique_words is considered as a word
    """
    freq = n_gram_count(sentence, n_gram_phrase)
    total_words = len(sentence.split())
    remove = 0
    for word in unique_words:
      freq_word = n_gram_count(sentence, word)
      word_count = freq_word * len(word.split())
      freq_word = word_count - freq_word
      remove += freq_word
    total_words -= remove
    return freq/total_words

def corpus_unique(corpus, n_gram_words):
  """ returns the total unique words in corpus """
  all_sentences = (' '.join(corpus))
  all_words = list(set(all_sentences.split()))
  all_words.extend(n_gram_words)
  
  return list(set(all_words))


### Weak Label

In [7]:
n_gram_words = ['wheat and rice','good for farmer','stand with','700 farmer',
               'so call','so called','salute to','free market','rich farmer',
                'we support','adani','ambani','khalistan','flag','victory',
                'loss','corporates','middleman','middle man','khalistani','godi',
                'jai jawan','jai kisan','no food','election','suicide','sikh',
                'farmer from punjab','monopoly','only 6','win','other state']


### Make Feature Vector

In [8]:
def WeakFeatures(lines, n_gram_words):

    F = np.zeros((len(lines), len(n_gram_words)))

    comments_list = list(lines)

    IDF_dict = IDF(lines, n_gram_words)
    corpus_words = corpus_unique(comments_list, n_gram_words)

    for i in range(len(comments_list)):
        for j in range(len(n_gram_words)):
            if n_gram_count(comments_list[i], n_gram_words[j]) !=0:
                F[i][j] = 1
            else:
                F[i][j] = 0
    return F

In [9]:

X_train, X_test, y_train, y_test = train_test_split(df_sample['tokenized_comment'],
                                                    df_sample['label'], test_size=0.2,
                                                    random_state=37,
                                                    stratify=df_sample['label'])


In [10]:
N_SAMPLES = 1000

classifiers = {
    "SVM": make_pipeline(StandardScaler(), svm.SVC(probability=True, kernel='rbf')),
    "KNN": KNeighborsClassifier(n_neighbors=19),
    "LR": make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', penalty='l2', C=0.05, max_iter=10000, random_state=2)),
    "MLP": make_pipeline(StandardScaler(), MLPClassifier(max_iter=1000, activation='logistic'))
}


### Classification

In [11]:
X_seed, X_expand, y_seed, y_expand = train_test_split(X_train,
                                                      y_train, test_size=0.99,
                                                      random_state=40,
                                                      stratify=y_train)


In [12]:
#print('model ', 'algorithm, ', 'batch_size, ', 'best_amount, ', 'expanded_size, ', 'outp_algorithm, ', 'c_accuracy')

algorithms = ['LR', 'SVM', 'KNN', 'MLP']
batch_sizes = [20]
best_amounts = [5]
big_results = []
big_confusion = []
mdl = model_2
    
#X = np.hstack((X_2, X))

for algorithm in algorithms:
    alg = classifiers[algorithm]

    for batch_size in batch_sizes:

        for best_amount in best_amounts:

            X_uncertain_exp_u, y_uncertain_exp_u, X_certain_exp_u, y_certain_exp_u = Expand_UW(mdl,
                                                                                                alg,
                                                                                                list(
                                                                                                    X_seed),
                                                                                                list(
                                                                                                    y_seed),
                                                                                                list(
                                                                                                    X_expand),
                                                                                                list(
                                                                                                    y_expand),
                                                                                                batch_size,
                                                                                                best_amount,
                                                                                                n_gram_words
                                                                                                )

            # Weak supervision
            F = WeakFeatures(X_uncertain_exp_u, n_gram_words)

            # transform
            X = np.stack([np.mean([mdl[w] for w in text.split()], 0) for text in X_uncertain_exp_u])

            # join them
            XF = np.hstack((X,F))

            # Weak supervision
            FT = WeakFeatures(X_test, n_gram_words)

            # transform
            XT = np.stack([np.mean([mdl[w] for w in text.split()], 0)
                         for text in X_test])

            # join them
            XFT = np.hstack((XT, FT))

            c_model = alg.fit(XF, y_uncertain_exp_u)
            y_predict = c_model.predict(XFT)
            c_accuracy = metrics.accuracy_score(y_test, y_predict)
            c_prfsw = list(precision_recall_fscore_support(y_test, y_predict,average='binary'))

            l1 = ['N=2', algorithm, batch_size, best_amount, len(X_uncertain_exp_u), c_accuracy]
            l1.extend(c_prfsw)

            big_results.append(l1)
            big_confusion.append(metrics.confusion_matrix(y_test, c_model.predict(XFT)))
            #print(md[1], ',', algorithm, ',', batch_size, ',', best_amount, ',', len(X_uncertain_exp_u), ',', outp_algorithm, ',', c_accuracy)


KeyboardInterrupt: 

In [ ]:
print(np.array(big_results))

[['N=2' 'LR' 20 5 666 0.7296466973886329 0.6937269372693727
  0.6690391459074733 0.6811594202898551 None]
 ['N=2' 'SVM' 20 5 666 0.7864823348694316 0.7448275862068966
  0.7686832740213523 0.756567425569177 None]
 ['N=2' 'KNN' 20 5 666 0.7127496159754224 0.6546052631578947
  0.708185053380783 0.6803418803418804 None]
 ['N=2' 'MLP' 20 5 666 0.7342549923195084 0.6928571428571428
  0.6903914590747331 0.6916221033868093 None]]


In [ ]:
print(np.array(big_confusion))


[[[287  83]
  [ 93 188]]

 [[296  74]
  [ 65 216]]

 [[265 105]
  [ 82 199]]

 [[284  86]
  [ 87 194]]]
